# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 8 Assignment: Building a Kaggle Submission File**

**Student Name: Rebekah Griesenauer**

# Assignment Instructions

For this assignment you will use the [**reg-30-spring-2018.csv**](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/reg-30-spring-2018.csv) dataset to train a neural network and [**reg-30-spring-2018-eval.csv
**](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/reg-30-spring-2018-eval.csv) to use as test to build a submission (similar to Kaggle).  The training code used for this assignment will be identical to [Assignmnent 4](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class4.ipynb) and you are encouraged to use your [Assignment 4](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class4.ipynb) code as a starting point.  Refer to [Module 8](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class8_kaggle.ipynb) for instructions on producing a Kaggle type submission file.  Please note, Module #8 provides an example of producing a classification (iris) submission file, you will need to convert this for 

The dataframe that you submit should have two columns: *id* and *target*.  The *id* column should matchup with the test data file.  The *target* column is your prediction.  It is unlikely that the mean of *target* will match exacly with mine.



# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #8 Sample Code

The following code provides a starting point for this assignment.

In [39]:
import os
import pandas as pd
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

key = "r7Th7Lhul2aj0C93lQNZkA9QZFuc6RW1YfPO4QM2"  # This is an example key and will not work.
file = '/Users/rebekahgriesenauer/Documents/AI/deep_neural_networks/assignments/assignment_griesenauer_class8.ipynb'

# Begin assignment
path = "../data/"

filename_read = os.path.join(path,"reg-30-spring-2018.csv")
eval_filename_read = os.path.join(path,"reg-30-spring-2018-eval.csv")
df = pd.read_csv(filename_read)
df_test = pd.read_csv(eval_filename_read)


In [29]:
ids = df['id']
df.drop('id',1,inplace=True)
#Check to see what columns have null values
missing_value_columns = df.columns[df.isna().any()].tolist()
#replace missing values with median
missing_median(df, 'width')

#find columns with numeric data
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
#remove "target" and "id" from numeric columns list
numeric_columns.remove('target')
#encode numeric columns as z-scores
for column in numeric_columns:
    encode_numeric_zscore(df, column, mean=None, sd=None)

#find columns with non-numeric data
text_columns = df.select_dtypes(include = object).columns.tolist()
#ecode text/category items to dummies
for text_col in text_columns:
    encode_text_dummy(df, text_col)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
x,y = to_xy(df, 'target')


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [41]:
df_t

,id,distance,height,landings,number,pack,age,usage,region,weight,item,volume,width,max,power,size
0,2000,5347,6825.004775,9730.734834,24908.548400,49191.12100,34039.58379,19017.146600,RE-10,4823,IT-5,37238,26656.973,28371,13253,1544
1,2001,4618,14837.297930,6871.178180,57163.903390,43577.77472,18602.82248,39325.105810,RE-7,2682,IT-9,30777,11908.167,30468,12101,1908
2,2002,6478,6733.764714,29808.258410,19062.035290,55022.15236,57920.27506,10843.454930,RE-F,876,IT-13,31877,26959.481,14976,18273,1105
3,2003,2435,6706.189108,13511.989940,4386.510786,38575.26678,51397.37838,38331.383710,RE-3,1662,IT-4,29215,36186.432,19560,22499,1295
4,2004,4838,11288.869260,24209.124020,37126.606060,40552.02217,25539.77485,7143.935253,RE-6,3198,IT-13,27132,11118.775,686,2599,1268
5,2005,3399,9436.737408,33029.630770,4118.988356,43230.96576,25067.98914,29511.613770,RE-E,3203,IT-1,63936,35783.126,27644,14876,1110
6,2006,5709,14490.036310,17228.088080,40343.142090,51746.80257,31027.00922,45469.331590,RE-F,728,IT-F,23060,35080.586,32760,15571,1289
7,2007,6140,9581.174832,28459.038030,8069.629705,47077.36598,38540.29683,40618.179840,RE-7,2285,IT-1,50746,21992.789,32320,5668,1157
8,2008,5644,17019.628360,25854.556670,30974.842820,45155.09006,33841.77363,10271.418100,RE-4,4362,IT-12,52814,24156.612,20955,15770,1301
9,2009,6986,17092.472110,25417.563880,61221.269580,33067.55551,29558.21114,40542.519450,RE-B,4343,IT-2,54579,26590.887,3425,20128,1013


In [30]:
model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(50, activation='relu')) # Hidden 2
model.add(Dense(25, activation='relu')) # Hidden 3
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=3,epochs=250)


Epoch 1/250
Epoch 2/250
Epoch 3/250
Epoch 4/250
Epoch 5/250
Epoch 6/250
Epoch 7/250
Epoch 8/250
Epoch 9/250
Epoch 10/250
Epoch 11/250
Epoch 12/250
Epoch 13/250
Epoch 14/250
Epoch 15/250
Epoch 16/250
Epoch 17/250
Epoch 18/250
Epoch 19/250
Epoch 20/250
Epoch 21/250
Epoch 22/250
Epoch 23/250
Epoch 24/250
Epoch 25/250
Epoch 26/250
Epoch 27/250
Epoch 28/250
Epoch 29/250
Epoch 30/250
Epoch 31/250
Epoch 32/250
Epoch 33/250
Epoch 34/250
Epoch 35/250
Epoch 36/250
Epoch 37/250
Epoch 38/250
Epoch 39/250
Epoch 40/250
Epoch 41/250
Epoch 42/250
Epoch 43/250
Epoch 44/250
Epoch 45/250
Epoch 46/250
Epoch 47/250
Epoch 48/250
Epoch 49/250
Epoch 50/250
Epoch 51/250
Epoch 52/250
Epoch 53/250
Epoch 54/250
Epoch 55/250
Epoch 56/250
Epoch 57/250
Epoch 58/250
Epoch 59/250
Epoch 60/250
Epoch 61/250
Epoch 62/250
Epoch 63/250
Epoch 64/250
Epoch 65/250
Epoch 66/250
Epoch 67/250
Epoch 68/250
Epoch 69/250
Epoch 70/250
Epoch 71/250
Epoch 72/250
Epoch 73/250
Epoch 74/250
Epoch 75/250
Epoch 76/250
Epoch 77/250
Epoch 78

In [31]:
ids = df_test['id']
df_test.drop('id',1,inplace=True)
#Check to see what columns have null values
missing_value_columns = df_test.columns[df_test.isna().any()].tolist()
#replace missing values with median
missing_median(df_test, 'width')

#find columns with numeric data
numeric_columns = df_test.select_dtypes(include=[np.number]).columns.tolist()
#encode numeric columns as z-scores
for column in numeric_columns:
    encode_numeric_zscore(df_test, column, mean=None, sd=None)

#find columns with non-numeric data
text_columns = df_test.select_dtypes(include = object).columns.tolist()
#ecode text/category items to dummies
for text_col in text_columns:
    encode_text_dummy(df_test, text_col)
    
x = df_test.as_matrix().astype(np.float32)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [33]:
pred = model.predict(x)

Final score (RMSE): 4.220114707946777


In [34]:
submit_df=pd.DataFrame(pred)
submit_df.insert(0,'id',ids)
submit_df.columns = ['id','target']

In [38]:
df_test

,distance,height,landings,number,pack,age,usage,weight,volume,width,...,item-IT-6,item-IT-7,item-IT-8,item-IT-9,item-IT-A,item-IT-B,item-IT-C,item-IT-D,item-IT-E,item-IT-F
0,-0.006474,-0.615671,-0.949752,-0.413967,0.552830,-0.268150,-1.007117,1.610320,-0.490915,0.117728,...,0,0,0,0,0,0,0,0,0,0
1,-0.517480,0.892810,-1.172341,1.385646,0.094329,-1.608039,0.965407,-0.052595,-0.907522,-2.313557,...,0,0,0,1,0,0,0,0,0,0
2,0.786321,-0.632849,0.613088,-0.740160,1.029111,1.804661,-1.801032,-1.455316,-0.836594,0.167596,...,0,0,0,0,0,0,0,0,0,0
3,-2.047693,-0.638041,-0.655418,-1.558947,-0.314278,1.238482,0.868886,-0.844829,-1.008240,1.688624,...,0,0,0,0,0,0,0,0,0,0
4,-0.363267,0.224744,0.177250,0.267711,-0.152815,-1.005922,-2.160369,0.348182,-1.142553,-2.443685,...,0,0,0,0,0,0,0,0,0,0
5,-1.371960,-0.123958,0.863841,-1.573873,0.066002,-1.046872,0.012217,0.352066,1.230581,1.622141,...,0,0,0,0,0,0,0,0,0,0
6,0.247276,0.827430,-0.366156,0.447170,0.761579,-0.529637,1.562199,-1.570268,-1.405117,1.506329,...,0,0,0,0,0,0,0,0,0,1
7,0.549393,-0.096765,0.508065,-1.353456,0.380177,0.122506,1.091004,-0.360945,0.380085,-0.651145,...,0,0,0,0,0,0,0,0,0,0
8,0.201713,1.303679,0.305331,-0.075512,0.223165,-0.285320,-1.856594,1.252262,0.513431,-0.294447,...,0,0,0,0,0,0,0,0,0,0
9,1.142412,1.317393,0.271315,1.612017,-0.764150,-0.657127,1.083655,1.237504,0.627238,0.106834,...,0,0,0,0,0,0,0,0,0,0


In [37]:
submit(source_file=file,data=submit_df,key=key,no=8)

Success: Submitted Assignment #8 for rebekah.griesenauer:
This is your first submission of this assignment.

